# Example of Equity processing within OSEM


In [1]:
import datetime
import logging
import os
import pandas as pd

from ConfigurationClass import Configuration
from CurvesClass import Curves
from EquityClasses import *
from ExportData import save_matrices_to_csv
from ImportData import get_EquityShare, get_settings, import_SWEiopa, get_Cash, get_Liability, \
    get_configuration
from PathsClasses import Paths
from TraceClass import tracer


Set up the base folder

In [2]:
base_folder = os.getcwd()  # Get current working directory

Most of the run settings are saved in the configuration file:

In [3]:
conf: Configuration
conf = get_configuration(os.path.join(base_folder, "ALM.ini"), os)

These lines of code just extract the absolute location of different files:

In [4]:
parameters_file = conf.input_parameters
cash_portfolio_file = conf.input_cash_portfolio
equity_portfolio_file = conf.input_equity_portfolio

The settings object holds data about file locations, information about the run settings and model parameters such as modelling date.

In [5]:
settings = get_settings(parameters_file)

The EquityShare object contains information about each equity position. This includes:
 * Asset_ID
 * NACE
 * Issuer
 * Issue_Date
 * Dividend_Yield
 * Frequency
 * Market_Price
 * Growth_Rate
  

In [6]:
equity_input_generator = get_EquityShare(equity_portfolio_file)
equity_input = {equity_share.asset_id: equity_share for equity_share in equity_input_generator}

EquitySharePortfolio object contains all EquityShare objects in a dictionary.

In [7]:
equity_portfolio = EquitySharePortfolio(equity_input)

import_SWEiopa() reads the necessary data about the current yield curve. One of these parameters (the ufr or ultimate forward rate) is necessary in the equity example as ufr is used in the Gordon growth formula to calculate the terminal value of the equity position. Inside OSEM, the parameters related to the yield curve are saved in the Curves object. 

In [8]:
[maturities_country, curve_country, extra_param, Qb] = import_SWEiopa(settings.EIOPA_param_file,
                                                                          settings.EIOPA_curves_file, settings.country)
# Curves object with information about term structure
curves = Curves(extra_param["UFR"] / 100, settings.precision, settings.tau, settings.modelling_date,
                settings.country)

In [9]:
import numpy as np
from ImportData import import_SWEiopa
from CurvesClass import Curves

# ultimate forward rate
ufr = extra_param["UFR"]/100

# Numeric precision of the optimisation
precision = float(settings.precision)

# Targeted distance between the extrapolated curve and the ultimate forward rate at the convergence point
tau = float(settings.tau) # 1 basis point


In [10]:
curves.SetObservedTermStructure(maturity_vec=curve_country.index.tolist(), yield_vec=curve_country.values)
curves.CalcFwdRates()

In [11]:
curves.ProjectForwardRate(settings.n_proj_years)

In [12]:
curves.CalibrateProjected(settings.n_proj_years, 0.05, 0.5, 1000)

In [13]:
desired_mat = np.array([0.7, 1.2, 1.3543])
print(curves.RetrieveRates(3, desired_mat, "Discount"))
    

   Discount
0  0.982068
1  0.969492
2  0.965650


The first computation step inside the OSEM equity preparation process is the identification of all the unique dates and divedend size amounts. The representation of assets in terms of individual cash flows on the time-line is one of the core principles of OSEM. This is done by two functions. One for dividend dates and another for terminal rates.

Both functions generate a list of dictionaries containing the date of a cash flow and the amount. Same is also true for the terminal amount calculation. 

#### Calculation of the dividend amount:

The dividend size is calculated using the dividend yield provided as input for each equity position. However the market value changes as time moves forward. To account for this, the growth rate and the time fraction between the modelling date and the date of the cash flow is used to calculate a future market value.

ToDo Formulas

The same logic is applied to the calculation of terminal rates. The additional complexity is that at least in theory, the equity has perpetual cash flows that extend well beyond the end of the modelling period. To account for this, the Gordon growth model is used to calculate the present value of all future cash flows as at the end of the modelling period.

In [14]:
def create_dividend_dates(obj, modelling_date, end_date):
    """
    Create the list of dictionaries containing dates at which the dividends are paid out and the total amounts for
    all equity shares in the portfolio, for dates on or after the modelling date but not after the terminal date.

    Parameters
    ----------
    self: EquitySharePortfolio class instance
        The EquitySharePortfolio instance with populated initial portfolio.
    :type modelling_date: datetime.date
        The date from which the dividend dates and values start.
    :type end_date: datetime.date
        The last date that the model considers (end of the modelling window).

    Returns
    -------
    :rtype all_dividends
        A list of dictionaries with datetime keys and cash-flow size values, containing all the dates at which the coupons are paid out.
    """
    all_dividends = []
    dividends: dict[date, float] = {}
    equity_share: EquityShare
    dividend_date: date
    for asset_id in obj.equity_share:
        equity_share = obj.equity_share[asset_id]  # Select one asset position
        dividends = create_single_equity(equity_share, modelling_date, end_date, equity_share.growth_rate)
        all_dividends.append(dividends)
    return all_dividends


In [15]:
test_share_1 = EquityShare(asset_id=1, nace='A1.4.5', issuer=None, issue_date=datetime.date(2021, 12, 3), dividend_yield=0.03, frequency=1, market_price=94.0, growth_rate=0.01)

In [16]:
test_dividends = test_share_1.create_single_cash_flows(settings.modelling_date, settings.end_date, 0.009831899404525758)
test_terminal = test_share_1.create_single_terminal(settings.modelling_date, settings.end_date, 0.009831899404525758)

In [17]:
def price_share(test_dividends, test_terminal, modelling_date, curves):
    Datefrac = []
    Values = []
    for key, value in test_dividends.items():
        date_frac = (key-modelling_date).days/365.25
        Datefrac.append(date_frac)
        Values.append(value)
        
    for key, value in test_terminal.items():
        date_frac = (key-modelling_date).days/365.25
        Datefrac.append(date_frac)
        Values.append(value)
    
    Datefrac = pd.DataFrame(data = Datefrac, columns = ["Date Fraction"]) # No need for Dataframes. Remove them
    Values = pd.DataFrame(data = Values, columns = ["Cash flow"])

    discount = curves.RetrieveRates(3, Datefrac.iloc[:, 0].to_numpy(), "Discount")

    nodisc_value = Values.values*discount
    disc_value = sum(nodisc_value.values)
    return disc_value

In [18]:
test_share_1.market_price

94.0

In [19]:
print(price_share(test_dividends, test_terminal, settings.modelling_date, curves)[0])

-213807.11370387784


In [20]:
test_share_1.bisection_growth(-1, 1, settings.modelling_date, settings.end_date, curves, 0.00000001,100000)

0.009999997913837433

In [21]:
dividend_dates = equity_portfolio.create_dividend_dates(settings.modelling_date, settings.end_date)
terminal_dates = equity_portfolio.create_terminal_dates(modelling_date=settings.modelling_date,
                                                            terminal_date=settings.end_date,
                                                            terminal_rate=curves.ufr)


All cash flows can be represented in a matrix with all possible cash flow dates as columns and all equities as rows. The non-zero entries then represent the value of the cash flow at that date. The first step is to calculate the unique dates for the entire portfolio of equities. This is done by the unique_dates_profiles() function.

The same logic can be applied to terminal dates. 

Both can then conveniently be represented as DataFrames.

Note that a vector of growth rates is also provided as output. This makes it simpler to increase the market value of the portfolio as OSEM moves from one modelling period to the next one.

In [22]:
unique_list = equity_portfolio.unique_dates_profile(dividend_dates)
unique_terminal_list = equity_portfolio.unique_dates_profile(terminal_dates)


In [23]:
[market_price_df, growth_rate_df] = equity_portfolio.init_equity_portfolio_to_dataframe(settings.modelling_date)


Cash is used in the trading algorithm to manage liquidity. The liquidity between buying/selling and the liquidity between modelling periods is saved in Cash. Therefore, for the Equity demonstration, Cash is also imported.

In [24]:
cash = get_Cash(cash_portfolio_file)

bank_account = pd.DataFrame(data=[cash.bank_account], columns=[settings.modelling_date])

The create_cashflow_dataframe() function converts the list of dictionaries of cashflows and dates, into a single DataFrame:

In [25]:
def create_cashflow_dataframe(cash_flow_dates, unique_dates) -> pd.DataFrame:
    cash_flows = pd.DataFrame(data=np.zeros((len(cash_flow_dates), len(unique_dates))),
                              columns=unique_dates)  # Dataframe of cashflows (columns are dates, rows, assets)
    counter = 0
    for asset in cash_flow_dates:
        keys = asset.keys()
        for key in keys:
            cash_flows[key].iloc[counter] = asset[key]
        counter += 1
    return cash_flows

In [26]:
cash_flows = create_cashflow_dataframe(dividend_dates, unique_list)
   
# Dataframe with terminal cash flows
terminal_cash_flows = create_cashflow_dataframe(terminal_dates, unique_terminal_list)
